In [ ]:
import arcpy
from arcpy import env
from collections import defaultdict
from  statistics import mean
import numpy
import time
start_time = time.time()

def createFishnet ():
    
    # set workspace environment
    # arcpy.env.workspace = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec"
    #env.workspace = r"C:\Users\ditsuhi\Desktop\arcMap\madrid"
    # Set coordinate system of the output fishnet
    #env.outputCoordinateSystem = arcpy.SpatialReference("ETRS_1989_UTM_Zone_30N")
    if arcpy.Exists(r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\fishnetAirMetTraf.shp"):
        arcpy.Delete_management(r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\fishnetAirMetTraf.shp")

    outFeatureClass = "fishnetAirMetTraf.shp"
    # Set the origin of the fishnet
    originCoordinate = '434215.234430378 4466449.72526323'
    # Set the orientation
    yAxisCoordinate = '434215.234430378 4466459.72526323'
    # Enter 0 for width and height - these values will be calcualted by the tool
    cellSizeWidth = '1000'
    cellSizeHeight = '1000'
    # Number of rows and columns together with origin and opposite corner 
    # determine the size of each cell 
    numRows =  '0'
    numColumns = '0'
    oppositeCoorner = '451215.234430378 4486449.72526323'
    # Create a point label feature class 
    labels = 'NO_LABELS'
    # Extent is set by origin and opposite corner - no need to use a template fc
    templateExtent = r'C:\Users\ditsuhi\Desktop\arcMap\madrid\fishnetMeteorStation.shp'
    # Each output cell will be a polygon
    geometryType = 'POLYGON'
    arcpy.CreateFishnet_management(outFeatureClass, originCoordinate, yAxisCoordinate, cellSizeWidth, cellSizeHeight, numRows, numColumns, oppositeCoorner, labels, templateExtent, geometryType)    


    
def makeFeatureLayerAndTableViewUsingArcpy (numFiles):
    
    #numFiles in arguments is the number of files 
    
    FishnetAirMetTraf= r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec\fishnetAirMetTraf.shp"
    metStation = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\meteo.shp"
    metdata = r"C:\Users\ditsuhi\Desktop\arcMap\trafficFishnet\gridFishnet\gridFishnet.gdb\metData_CopyRowsFinal"
    airQualityPoint = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\airStation.shp"
    no2data = r"C:\Users\ditsuhi\Desktop\arcMap\trafficFishnet\gridFishnet\gridFishnet.gdb\NO2_CopyRows"
    arcpy.management.MakeFeatureLayer(FishnetAirMetTraf, "FishnetAirMetTraf")
    arcpy.management.MakeFeatureLayer(airQualityPoint, "airQualityPoint")
    arcpy.MakeTableView_management(no2data, "no2data")
    arcpy.management.MakeFeatureLayer(metStation, "metStation")
    arcpy.MakeTableView_management(metdata, "metdata")
    
    return FishnetAirMetTraf, metStation, metdata, airQualityPoint, no2data
    
        
def addFieldsToFishnet (metMagnitude):
    arcpy.management.AddField(FishnetAirMetTraf, "NO2", "Double")
    for metMagnitudeKey, metMagnitudeValue  in metMagnitude.items():        
        arcpy.management.AddField(FishnetAirMetTraf, metMagnitudeValue, "Double")
  
    
def spatialJoin (FishnetAirMetTraf, metStation, metdata, airQualityPoint):
    #def spatialJoin (FishnetAirMetTraf, TrafficStation, metStation, metdata, airQualityPoint):
    
    outSpatialAir = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\spatialTargetAir"
    arcpy.analysis.SpatialJoin(FishnetAirMetTraf, airQualityPoint, outSpatialAir)
    outSpatialTargetAir = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\spatialTargetAir.shp"
    arcpy.management.MakeFeatureLayer(outSpatialTargetAir, "outSpatialTargetAir")
    with arcpy.da.SearchCursor(outSpatialTargetAir, ["Join_Count", "FID", "CODIGO_COR"]) as outSpatial_cursor:   
        station_FID_Code_dict  = {row[1]: row[2] for row in outSpatial_cursor if row[0] == 1}
    
    outSpatialMet = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\spatialTargetMeteor"
    arcpy.analysis.SpatialJoin(FishnetAirMetTraf, metStation, outSpatialMet)
    outSpatialTargetMet = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\spatialTargetMet.shp"
    arcpy.management.MakeFeatureLayer(outSpatialTargetMet, "outSpatialTargetMet")
    with arcpy.da.SearchCursor(outSpatialTargetMet, ["Join_Count", "FID", "CÓDIGO_CO"]) as outSpatial_cursor:     
        station_FID_Code_dict_Met  = {row[1]: row[2] for row in outSpatial_cursor if row[0] == 1}        
  
    return station_FID_Code_dict, station_FID_Code_dict_Met


def airDataMatchedByStation (FishnetAirMetTraf, station_FID_Code_dict, hour, no2data, currentDate):    
    with arcpy.da.UpdateCursor(FishnetAirMetTraf, ["FID", "NO2"]) as fishLabel_cursor:
        for x in fishLabel_cursor:
            for key, value in station_FID_Code_dict.items():
                if x[0] == key:                                
                    with arcpy.da.SearchCursor(no2data, ["Fecha", "ESTACION", f"H0{hour}" if hour < 10 else f"H{hour}"]) as no2data_cursor:                    
                        for fecha in no2data_cursor:                    
                            if str(fecha[0].date()) == currentDate and fecha[1] == value:                
                                x[1] = fecha[2]
                                fishLabel_cursor.updateRow(x)
    


def metDataMatchedByStation (metMagnitude, FishnetAirMetTraf, station_FID_Code_dict_Met, metdata, hour, currentDate):    
    for  metMagnitudeKey, metMagnitudeValue  in metMagnitude.items():    
        with arcpy.da.UpdateCursor(FishnetAirMetTraf, ["FID", metMagnitudeValue]) as fishLabel_cursor:
            for x in fishLabel_cursor:
                for key, value in station_FID_Code_dict_Met.items():
                    if x[0] == key:                                
                        with arcpy.da.SearchCursor(metdata, ["Fecha", "ESTACION", "MAGNITUD", f"H0{hour}" if hour < 10 else f"H{hour}"]) as metdata_cursor:                    
                            for fecha in metdata_cursor:
                                if str(fecha[0].date()) == currentDate and fecha[1] == value and fecha[2] == int(metMagnitudeKey): 
                                    x[1] = fecha[3]
                                    fishLabel_cursor.updateRow(x)
                                
    

def exportAttributeTableToCSVFile(cols, num, currentDate):
    nparr = arcpy.da.FeatureClassToNumPyArray(FishnetAirMetTraf, cols)   
    numpy.savetxt(fr'C:\Users\ditsuhi\Desktop\arcMap\trafficCSVFiles\fishnetAirMet{currentDate.year}_{currentDate.strftime("%b").lower()}_{currentDate.day}_{num}.csv', nparr, delimiter=",", fmt="%s", header = "FID, NO2, UV, windSpeed, windDir, Temp, Humidity, Pressure, SolarRad, Prec")
    
    

arcpy.env.workspace = r"C:\Users\ditsuhi\Desktop\arcMap\madrid\traf2020dec"  
metMagnitude = {"80": "UV", "81": "windSpeed", "82": "windDir", "83": "Temp", "86": "Humidity", "87": "Pressure", "88": "SolarRad", "89": "Prec"}
trafColumn = ["intensidad", "ocupacion", "carga", "vmed"]
#allColumns = ["FID", "NO2", "UV", "windSpeed", "windDir", "Temp", "Humidity", "Pressure", "SolarRad", "Prec", "intensidad", "ocupacion", "carga", "vmed"]
allColumns = ["FID", "NO2", "UV", "windSpeed", "windDir", "Temp", "Humidity", "Pressure", "SolarRad", "Prec"]


from datetime import timedelta, date


end_date = date(2019, 1, 1)
start_date = date(2019, 1, 1)
delta = timedelta(days=-1)

while end_date >= start_date:
    print(str(end_date)) 
    for itemNum in range(12, 13):
        createFishnet ()
        FishnetAirMetTraf, metStation, metdata, airQualityPoint, no2data = makeFeatureLayerAndTableViewUsingArcpy(itemNum)
        station_FID_Code_dict, station_FID_Code_dict_Met = spatialJoin (FishnetAirMetTraf, metStation, metdata, airQualityPoint)        
        addFieldsToFishnet (metMagnitude)
        airDataMatchedByStation (FishnetAirMetTraf, station_FID_Code_dict, itemNum, no2data, str(end_date))       
    end_date += delta

print(f"{(time.time() - start_time)} second")  